In [39]:
# -------------------------------------------------------------#
#           Step 0, import libraries                          #
# -------------------------------------------------------------#
import geopandas as gpd
import rasterio
from rasterio import mask
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm


In [40]:

# -------------------------------------------------------------#
#           Step 1, import the data                           #
# -------------------------------------------------------------#
GMBA = {"path": "./data/GMBA/GMBA_Inventory_v2.0_Level_03.shp"}

DEM = {"path": "./data/DEM/GMTED2010_30.tiff"}

TREE_LINE_TEMP = {
    "path": "./data/temp_treeline/Alpine_Biome_Suzette.shp"
}  # temp_treeline values are in column: "MEAN_Avg_c", names of mountain ranges are in column "MapName"

MARGIN = 0.25  # margin for the treeline temperature

# Models (using the orininal data, not resampled)
CHELSA_MODEL = {"path": Path("./data/CHELSA/CHELSA_TraCE21k_bio01_-190_V1.0.tif")}
BEYER_MODEL = {"path": Path("./data/Beyer/LateQuaternary_Environment_-21020.tif")}
PGEM_MODEL = {"path": Path("./data/PALEO-PGEM/PALEO-PGEM-Series_bio1_mean_-21020.tif")}
WORLDCLIM_MODEL = {"path": Path("./data/Worldclim1/cclgmbi1_21ka_2.5.tif")} #these values need to be multiplied by 10 to get normal celcius values
ECOCLIMATE_MODEL = {"path": Path("./data/ecoclimate/bio#baseline_Modern(1950-1999)#CCSM_LGM(21ka)_bio1.tif")}
GGC_MODEL = {"path": Path("./data/ggc/ggc2.tif")}

#do not use
#LR_PRESENT = {"path": Path("./data/CHELSA/CHELSA_TraCE21k_bio01_20_V1.0.tif")}


In [ ]:
# Check dimensions of the rasters CHELSA_MODEL, BEYER_MODEL, PGEM_MODEL, WORLDCLIM_MODEL, ECOCLIMATE_MODEL, GGC_MODEL
with rasterio.open(CHELSA_MODEL["path"]) as src:
    print("chelsa",src.width, src.height)
with rasterio.open(BEYER_MODEL["path"]) as src: 
    print("beyer",src.width, src.height)        
with rasterio.open(PGEM_MODEL["path"]) as src:
    print("paleo-pgem",src.width, src.height)
with rasterio.open(WORLDCLIM_MODEL["path"]) as src:
    print("worldclim",src.width, src.height)
with rasterio.open(ECOCLIMATE_MODEL["path"]) as src:
    print("ecoclimate",src.width, src.height)
with rasterio.open(GGC_MODEL["path"]) as src:
    print("ggc",src.width, src.height)


In [ ]:
# load and test dimensions of the resampled files
BEYER_RES_MODEL = {"path": Path("./data/Beyer/resampled/beyer_lgm_resampled.tif")}
with rasterio.open(BEYER_RES_MODEL["path"]) as src: 
    print("beyer res",src.width, src.height)  

PGEM_RES_MODEL = {"path": Path("./data/PALEO-PGEM/resampled/paleopgem_lgm_resampled.tif")}
with rasterio.open(PGEM_RES_MODEL["path"]) as src:
    print("paleo-pgem res",src.width, src.height)

WORLDCLIM_RES_MODEL = {"path": Path("./data/Worldclim1/resampled/worldclim_lgm_resampled.tif")}
with rasterio.open(WORLDCLIM_RES_MODEL["path"]) as src:
    print("worldclim res",src.width, src.height)

ECOCLIMATE_RES_MODEL = {"path": Path("./data/ecoclimate/resampled/ecoclimate_lgm_resampled.tif")}
with rasterio.open(ECOCLIMATE_RES_MODEL["path"]) as src:
    print("ecoclimate res",src.width, src.height)

In [41]:

# -------------------------------------------------------------#
#   Step 2, create the right variables for the loop            #
# -------------------------------------------------------------#

# DEM must be added to lgm raster of pgem model
# extract the elevation value from the dem where the lgm temp of pgem equals the treeline, temp +- the margin, for each mountain range
# save the extracted elevation value in a new raster, attach the name of the mountain range
# save the raster in a new folder   ./data/elevation_treeline

# Read the shapefile
mountain_ranges = gpd.read_file(GMBA["path"])

# Read the temperature shapefile
treeline_temp = gpd.read_file(TREE_LINE_TEMP["path"])

# Read the DEM
dem = rasterio.open(DEM["path"])

# Read the model
temperature_raster = rasterio.open(LR_PRESENT["path"]) #NOTE: CHANGE THIS TO THE MODEL YOU WANT TO USE

# If the temperature raster is smaller than the dem, resample the dem to the width of the temperature raster
if temperature_raster.width != dem.width:
    # NOTE: THIS ASSUMES SAME PROJECTION FOR ALL RASTERS
    # Scale to same width as the temperature raster, and keep the aspect ratio
    dem_scale = int(dem.width / temperature_raster.width)
    new_dem_height = int(dem.height / dem_scale)
    new_dem_width = int(dem.width / dem_scale)
    dem_downsampled = np.zeros((new_dem_height, new_dem_width), dtype=dem.read(1).dtype)
    dem_downsampled, new_dem_transform = rasterio.warp.reproject(source=dem.read(),
                                destination=dem_downsampled,
                                src_transform=dem.transform,
                                dst_transform=temperature_raster.transform,
                                src_crs=dem.crs,
                                dst_crs=dem.crs,
                                dst_nodata=dem.nodata,
                                resampling=rasterio.enums.Resampling.average)
    
    # resize dem_downsampled to the size of the temperature raster
    dem_downsampled = dem_downsampled[:temperature_raster.height, :temperature_raster.width]
    
    new_dem_profile = dem.profile
    new_dem_profile.update(transform=new_dem_transform, driver='GTiff',
                            height=temperature_raster.height, width=temperature_raster.width)

    with rasterio.open("./data/dem_downsampled.tif", "w", **new_dem_profile) as dst:
        dst.write(dem_downsampled, 1)
    
    # Overwrite the dem object with the downsampled dem
    dem = rasterio.open("./data/dem_downsampled.tif")
    

### FILE BELOW WAS SPLIT INTO TWO LOOPS BECAUSE OF MEMORY CONCERNS
Running time Chelsa and ggc was 67 min, Worldclim 2.5 minutes, other models < 1 minute.

In [42]:
# -------------------------------------------------------------#
#   Step 3, masking, padding, extraction, saving               #
# -------------------------------------------------------------#

# create an empty mask the size of the dem
global_dem_mask = np.full((dem.height, dem.width), False, dtype=bool)

global_meta = dem.meta.copy()
global_meta.update(
    {
        "driver": "GTiff",
        "nodata": -999,
        "compress": "lzw",
    }
)
temperature_not_found = []

print("Calculating the elevation for the biome")
# Loop over the mountain ranges
for idx, mountain_range in tqdm(list(mountain_ranges.iterrows())):
    # Get the name of the mountain range
    mountain_range_name = mountain_range["MapName"]
    mountain_range_temperature = treeline_temp["MEAN_Avg_c"][treeline_temp["MapName"] == mountain_range_name]
    # NOTE: THIS CHECK IS HERE TO STOP THE CODE FROM BREAKING WHEN NO TEMPERATURE IS FOUND
    if mountain_range_temperature.empty:
        temperature_not_found.append(mountain_range_name)
        continue

    # Mask the CHELSA model with the mountain range geometry
    temperature_raster_within_mountain_range, temperature_raster_within_mountain_range_transform = mask.mask(
        temperature_raster, [mountain_range["geometry"]], nodata=-999, all_touched=True)



    # Set mask to True where the temperature is below the treeline temperature
    global_dem_mask = np.logical_or(global_dem_mask, np.where(np.logical_and(temperature_raster_within_mountain_range[0] <= float(mountain_range_temperature), temperature_raster_within_mountain_range[0] != -999),
                     True,
                        False))


# # Save the global dem raster for the biome NOTE:>>>CHANGE NAME<<<
with rasterio.open("./data/elevation_treeline_biome/elevation_biome_lr_present.tif", "w", **global_meta) as dst:
    dst.write(np.where(global_dem_mask, dem.read(1), -999), 1)


print("Calculating the elevation for the treeline")
# Reset the global mask
global_dem_mask = False
# Loop over the mountain ranges
for idx, mountain_range in tqdm(list(mountain_ranges.iterrows())):
    # Get the name of the mountain range
    mountain_range_name = mountain_range["MapName"]
    mountain_range_temperature = treeline_temp["MEAN_Avg_c"][treeline_temp["MapName"] == mountain_range_name]
    # NOTE: THIS CHECK IS HERE TO STOP THE CODE FROM BREAKING WHEN NO TEMPERATURE IS FOUND
    if mountain_range_temperature.empty:
        continue

    # Mask the CHELSA model with the mountain range geometry
    temperature_raster_within_mountain_range, temperature_raster_within_mountain_range_transform = mask.mask(
        temperature_raster, [mountain_range["geometry"]], nodata=-999, all_touched=True)

    # create masked array for chelsa_within_mountain_range
    masked_temperature_raster = np.ma.masked_array(temperature_raster_within_mountain_range[0], temperature_raster_within_mountain_range[0] == -999)

    # Find the mask for the treeline 
    global_dem_mask = np.logical_or(global_dem_mask, np.where(np.logical_and(masked_temperature_raster <= float(mountain_range_temperature) + MARGIN, masked_temperature_raster >= float(mountain_range_temperature) - MARGIN),
        True,
        False,
    ))
                                    
# Save the global dem raster for the treeline NOTE:>>>CHANGE NAME<<<
with rasterio.open("./data/elevation_treeline_biome/elevation_treeline_lr_present.tif", "w", **global_meta) as dst:
    dst.write(np.where(global_dem_mask, dem.read(1), -999), 1)
    
# Report on the mountain ranges where the temperature was not found
print("NO TEMPERATURES FOUND FOR FOLLOWING MOUNTAIN RANGE NAMES:", "\n".join(temperature_not_found))
    

Calculating the elevation for the biome


  0%|          | 0/178 [00:00<?, ?it/s]C:\Users\elren9761\AppData\Local\Temp\ipykernel_15464\3533603069.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  global_dem_mask = np.logical_or(global_dem_mask, np.where(np.logical_and(temperature_raster_within_mountain_range[0] <= float(mountain_range_temperature), temperature_raster_within_mountain_range[0] != -999),
  1%|          | 1/178 [00:26<1:18:09, 26.50s/it]C:\Users\elren9761\AppData\Local\Temp\ipykernel_15464\3533603069.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  global_dem_mask = np.logical_or(global_dem_mask, np.where(np.logical_and(temperature_raster_within_mountain_range[0] <= float(mountain_range_temperature), temperature_raster_within_mountain_range[0] != -999),
  1%|          | 2/178 [00:47<1:08:27, 23.34s/it]C:\Users\el

Calculating the elevation for the treeline


  0%|          | 0/178 [00:00<?, ?it/s]C:\Users\elren9761\AppData\Local\Temp\ipykernel_15464\3533603069.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  global_dem_mask = np.logical_or(global_dem_mask, np.where(np.logical_and(masked_temperature_raster <= float(mountain_range_temperature) + MARGIN, masked_temperature_raster >= float(mountain_range_temperature) - MARGIN),
  1%|          | 1/178 [00:24<1:13:36, 24.95s/it]C:\Users\elren9761\AppData\Local\Temp\ipykernel_15464\3533603069.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  global_dem_mask = np.logical_or(global_dem_mask, np.where(np.logical_and(masked_temperature_raster <= float(mountain_range_temperature) + MARGIN, masked_temperature_raster >= float(mountain_range_temperature) - MARGIN),
  1%|          | 2/178 [00:48<1:11:36,

NO TEMPERATURES FOUND FOR FOLLOWING MOUNTAIN RANGE NAMES: Plateau of Mozambique
Coro Region
Venezuelan Coastal Range
Guiana Highlands
Tumucumaque Uplands
Atlantic Plateau (Brazil)
Eastern Arc Mountains
Great Dividing Range
Guinea Highlands
Blue Ridge Mountains
Central America Volcanic Arc
Sahara Ranges
Indian Subcontinent Ranges
Korean Peninsula
Arabian Peninsula
Levant Ranges
Great Plains
U.S. Interior Highlands
Fiji
Marquesas Islands
New Caledonia
Vanuatu Islands
Solomon Islands
Society Islands
Lesser Antilles
Khentei-Daur Highlands
Yankan - Tukuringra - Soktakhan - Dzhagdy group of mountain ranges
Central Highland
Maranhão-Piauí Highlands
Southern Highland
Northeastern Highland
Horn of Africa Highlands
Macaronesian Islands
Saint Helena
Ascension Island
Tristan da Cunha
Amsterdam Island
Comoro Islands
Seychelles
Andaman and Nicobar Islands
Flinders Lofty Block
Central Ranges
West Coast Ranges
Northern Ranges
Western Lowlands
Eastern Escarpment (Madagascar)
Central Plateau
Appalachian

In [ ]:
# Plot the treeline_extraction #>>> CHANGE FILE NAME <<< 
with rasterio.open("./data/elevation_treeline_biome/elevation_biome_chelsa_present.tif") as src: 
    plt.imshow(src.read(1)[::-1], cmap="terrain")
    # plot the mountain ranges
    plt.colorbar()
    plt.show()

In [ ]:
# Plot the treeline_extraction upside down #>>> CHANGE FILE NAME <<< 
with rasterio.open("./data/elevation_treeline_biome/elevation_biome_chelsa_present.tif") as src: 
    plt.imshow(src.read(1)[::-1], cmap="terrain")
    # plot the mountain ranges
    plt.colorbar()
    plt.show()

In [ ]:
#list the mountain ranges that have temperature
for temp in treeline_temp.iterrows():
    print(temp[1]["MapName"], temp[1]["MEAN_Avg_c"])